# First NGSolve example (Unit 1.1)

Let us solve the Poisson problem.
$$\text{find: } u \in H_{0,D}^1 \int_{\Omega} \nabla u \nabla v = \int_\Omega f v \hspace{2em} \forall v \in H_{0,D}^1.$$
## Quick steps to solution:
#### 1. Import NGSolve and Netgen Python modules:


In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square

# these lines removed in new versions
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()


**Note (DD):** to generate a regular mesh with the boundary condition names set correctly
 just remove the maxh value from unit_square.GenerateMesh, then in Netgen refine it a couple times and save it.
 
 **Note (DD):** We can load geometry files but I'm not sure how to save them.

#### 2. Generate an unstructured mesh

In [2]:

mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))#maxh=0.25))
#print(unit_square.SegmentData())
#mesh = Mesh('dow_test7.vol')
#mesh.Boundaries('top')

In [3]:
mesh.nv, mesh.ne

(39, 56)

- Here we prescribed a maximal mesh-size of 0.2 using the maxh flag.

- The mesh can be viewed by switching to the Mesh tab in the Netgen GUI.

#### 3. Declare a finite element space:

In [4]:

fes = H1(mesh, order=2, dirichlet="bottom|top|left|right")
fes.ndof  # number of unknowns in this space


133

Python's help system displays further documentation.

In [5]:
help(fes)

Help on H1 in module ngsolve.comp object:

class H1(FESpace)
 |   Keyword arguments can be:
 |  order: int = 1
 |    order of finite element space
 |  complex: bool = False
 |  dirichlet: regexpr
 |    Regular expression string defining the dirichlet boundary.
 |    More than one boundary can be combined by the | operator,
 |    i.e.: dirichlet = 'top|right'
 |  definedon: Region or regexpr
 |    FESpace is only defined on specific Region, created with mesh.Materials('regexpr')
 |    or mesh.Boundaries('regexpr'). If given a regexpr, the region is assumed to be
 |    mesh.Materials('regexpr').
 |  dim: int = 1
 |    Create multi dimensional FESpace (i.e. [H1]^3)
 |  dgjumps: bool = False
 |    Enable discontinuous space for DG methods, this flag is needed for DG methods,
 |    since the dofs have a different coupling then and this changes the sparsity
 |    pattern of matrices.
 |  
 |  Method resolution order:
 |      H1
 |      FESpace
 |      pybind11_builtins.pybind11_object
 |    


#### 4. Declare test function, trial function, and grid function

- Test and trial function are symbolic objects - called ProxyFunctions - that help you construct bilinear forms (and have no space to hold solutions).

- GridFunctions, on the other hand, represent functions in the finite element space and contains memory to hold coefficient vectors.



In [6]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
gfu = GridFunction(fes)  # solution 

A shorter command for obtaining at once both trial and test variables is now available:

In [7]:
u, v = fes.TnT()

#### 5. Define and assemble linear and bilinear forms:

In [8]:
a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(x*v)
f.Assemble()


In [9]:
print(f.vec)

 0.00666667
 0.00964013
 0.00666667
 0.0114203
 0.0239003
 0.0231589
 0.0187204
 0.0151473
 0.0163802
 0.0183837
 0.0162275
 0.0192201
 0.0245988
 0.0155399
 0.0175438
 0.0172541
 0.0141324
 0.0143048
 0.0210066
 0.0234006
 0.0565405
 0.0422274
 0.0348701
 0.0245431
 0.0410306
 0.0329496
 0.0338671
 0.0246024
 0.0334822
 0.0353989
 0.0310422
 0.0220977
 0.0361961
 0.0383175
 0.0425847
 0.0312368
 0.0342741
 0.026937
 0.0444888
 -0.000833333
 -0.000833333
 -0.000601784
 -0.000603232
 -0.00120502
 -0.000833333
 -0.000833333
 -0.000726751
 -0.000700785
 -0.00142754
 -0.00101667
 -0.00197087
 -0.00215421
 -0.000821193
 -0.00207368
 -0.0018782
 -0.000725239
 -0.00161481
 -0.00151886
 -0.00129163
 -0.00116817
 -0.000814448
 -0.00123308
 -0.00144429
 -0.00068386
 -0.0016141
 -0.00148352
 -0.000663325
 -0.00136512
 -0.00134458
 -0.00173918
 -0.00156917
 -0.00062762
 -0.00161389
 -0.00133566
 -0.000745979
 -0.00119651
 -0.00131487
 -0.000766081
 -0.0014269
 -0.001447
 -0.00143001
 -0.00139068
 

I tried to get NGSolve to generate a matrix similar to the one in Claes Johnson 1.4, but it always puts in rows for the boundary elements and the diagonal elements have a different pattern.  

This is because the matrix in Claes Johnson doesn't include elements corresponding to Dirichlet boundary conditions, whereas NGSolve assumes Neumann boundary conditions when constructing the matrix and only applies the Dirichlet conditions when the matrix is inverted.

In [10]:
print(a.mat)

Row 0:   0: 1
Row 1:   1: 0.829127
Row 2:   2: 1
Row 3:   3: 0.880753
Row 4:   0: -0.5   4: 1.90755
Row 5:   4: -0.559177   5: 1.91948
Row 6:   5: -0.37742   6: 1.7799
Row 7:   1: -0.222678   6: -0.298266   7: 1.83885
Row 8:   1: -0.223343   8: 1.90243
Row 9:   8: -0.375377   9: 1.75223
Row 10:   9: -0.283322   10: 1.78161
Row 11:   2: -0.5   10: -0.268388   11: 1.99954
Row 12:   2: -0.5   11: -0.098129   12: 1.84322
Row 13:   12: -0.211616   13: 1.75354
Row 14:   13: -0.319115   14: 1.77273
Row 15:   3: -0.359363   14: -0.356559   15: 1.85161
Row 16:   3: -0.354152   16: 1.80192
Row 17:   16: -0.146234   17: 1.85831
Row 18:   17: -0.280268   18: 1.8809
Row 19:   0: -0.5   4: -0.159147   18: -0.497592   19: 1.89108
Row 20:   4: -0.689223   5: -0.134026   18: -0.213492   19: -0.734339   20: 3.5107
Row 21:   5: -0.848855   6: -0.338846   20: -0.607596   21: 3.57058
Row 22:   6: -0.76537   7: -0.325948   21: -0.590849   22: 3.53156
Row 23:   1: -0.383106   7: -0.991955   8: -1.08169   22:

#### 6. Solve the system:

In [11]:
#gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse='umfpack')*f.vec
#gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse='sparsecholesky')*f.vec
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu)


The Dirichlet boundary condition constrains some degrees of freedom. The argument ```fes.FreeDofs()``` indicates that only the remaining "free" degrees of freedom should participate in the linear solve.

You can examine the coefficient vector of solution if needed:


In [12]:
print(gfu.vec)

       0
       0
       0
       0
       0
       0
       0
       0
 0.0734498
 0.109414
 0.109415
 0.0734809
       0
       0
       0
       0
       0
       0
       0
       0
 0.0480287
 0.0613986
 0.0628709
 0.0563951
 0.0996634
 0.111629
 0.0866537
 0.0563395
 0.059725
 0.0509623
 0.0312483
 0.0363499
 0.0499572
 0.0950059
 0.0820393
 0.0948911
 0.0684972
 0.107498
 0.0928058
       0
       0
       0
 -0.0386977
 -0.0218516
 -0.0385598
       0
       0
       0
 0.037502
       0
 -0.110535
 -0.0330141
       0
 -0.0884939
 -0.0263252
       0
 -0.0477798
 -0.0249668
 -0.0337592
 -0.0193756
 -0.0364835
 -0.00379618
 -0.0208124
 -0.0356349
 -0.00756482
 -0.020697
 -0.0363716
 -0.00621473
 -0.019954
 -0.0371197
 -0.00536581
       0
 -0.0623304
 -0.0252593
       0
 -0.0180479
 -0.0353573
       0
 -0.0180012
 -0.0369938
 -0.00259116
 -0.0304726
       0
 -0.0262815
 0.000375514
       0
 -0.0195198
 -0.0229837
       0
 -0.0685234
 -0.0213655
 -0.0237983
 -0.0251861
 -0.


## Ways to interact with NGSolve

- A jupyter notebook (like this one) gives you one way to interact with NGSolve. When you have a complex sequence of tasks to perform, the notebook may not be adequate.

- You can write an entire python script in a text editor and call python on the command line. (A script of the above is provided in poisson.py.)

      python3 poisson.py

- If you want the Netgen GUI, then use netgen on the command line:

      netgen poisson.py

- You can then ask for a python shell from the GUI's menu options (Solve -> Python shell).

